In [1]:
import cv2
import mediapipe 
import numpy as np
import time
from PIL import Image
import os
import img2pdf
import glob


########################################

# FOLDER CREATION

########################################


fp = "paint"

if os.path.exists(fp):
    pass
else:
    os.mkdir(fp)


########################################

# VIDEO CAPTURING

########################################


frames_per_second = 18.0
video_file = "recording.mp4"
video_type = cv2.VideoWriter_fourcc(*"XVID")
video = cv2.VideoCapture(0)
out = cv2.VideoWriter(video_file,video_type,frames_per_second,(640,480))
rec_count = False


drawingModule = mediapipe.solutions.drawing_utils
handsModule = mediapipe.solutions.hands
hands = handsModule.Hands(min_detection_confidence=0.7,min_tracking_confidence=0.7,max_num_hands=1)


img = cv2.imread("tool-removebg-preview.bmp")
img = img.astype('uint8')


mask = np.ones((480,640))*255
mask = mask.astype('uint8')





########################################

# GET TOOL FUNCTION

########################################


#get tools function
def getTool(x):
    if x < 50 + ml:
        return "record"

    elif x<100 + ml:
        return "line"

    elif x < 150 + ml:
        return"circle"

    elif x<200 + ml:
        return "rectangle"

    elif x<250 + ml:
        return "draw"
    
    elif x<300+ml:
        return "erase"

    elif x<350+ml:
        return "clear"

    elif x<400+ml:
        return "save"

    elif x<450+ml:
        return "pause"

    elif x<500+ml:
        return "exit"



def index_raised(yi, y9):
    
    if (y9 - yi) > 40:
        return True

    return False




########################################

# VARIABLE DECLARATION

########################################


curr_tool = "Select"

ml = 0
max_x,max_y = 500+ml,50
time_init = True
rad = 30
thick = 4
var_inits = False
prevx,prevy = 0,0
fileNum = 0
x_point,y_point = 640,480
exit_variable = False
rec_count = False


########################################

# FRAME SEQUENCING / VIRTUAL WRITING

########################################


while (True):
    
    ret,frame = video.read()
    
    frame = cv2.flip(frame,1)
    
    rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

    results = hands.process(rgb)

    """ MASKING OF FRAME & MASK WINDOW """   
    
    frame = cv2.bitwise_and(frame, frame, mask=mask)
    frame[:max_y,ml:max_x] = cv2.addWeighted(img,0.7,frame[:max_y,ml:max_x],0.3,0)



    
 
    if results.multi_hand_landmarks != None:
        for i in results.multi_hand_landmarks:
            drawingModule.draw_landmarks(frame, i, handsModule.HAND_CONNECTIONS)

            x,y = int(i.landmark[8].x*640),int(i.landmark[8].y*480)

            if x<max_x+50 and y<max_y and x>ml:

                if time_init:

                    ctime = time.time()
                    time_init=False
                ptime = time.time()
                cv2.circle(frame,(x,y),rad,(0,255,255),3)
                rad-=1

                if (ptime-ctime)>0.8:
                    curr_tool = getTool(x)
                    time_init = True
                    rad = 40
            
            else:
                time_init = True
                rad = 40

            if curr_tool == "clear":

                xi,yi = int(i.landmark[12].x*640),int(i.landmark[12].y*480)
                y9 = int(i.landmark[9].y*480)

                if (index_raised(yi,y9)):

                    if not(var_inits):
                        var_inits = True
                    mask = np.ones((480,640))*255
                    mask = mask.astype('uint8')

                else:
                    if var_inits:
                        var_inits = False

            elif curr_tool == "record" and rec_count == False:
                rec_count = True
                out.write(frame)
                
            elif curr_tool == "pause":

                xi,yi = int(i.landmark[12].x*640),int(i.landmark[12].y*480)
                y9 = int(i.landmark[9].y*480)

                if (index_raised(yi,y9)):
                    while (True):
                        
                            cv2.putText(frame,"Video paused.. Press \'p\' to Play !!", (120,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                            cv2.imshow("Test video",frame)
                            cv2.imshow("mask",mask)
                            key2 = cv2.waitKey(0)

                            if (key2==ord('p')):
                                curr_tool = "Select"
                                break
                            

                else:

                    cv2.putText(frame,"Raise your control finger !!", (120,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)


            elif curr_tool == "exit":

                xi,yi = int(i.landmark[12].x*640),int(i.landmark[12].y*480)
                y9 = int(i.landmark[9].y*480)

                if (index_raised(yi,y9)):

                    cv2.putText(frame,"EXITING !!", (170,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                    out.release()
                    exit_variable = True
                    break

                else:

                    cv2.putText(frame,"Raise your control finger !!", (120,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

                
            
            elif curr_tool == "line":
                
                xi,yi = int(i.landmark[12].x*640),int(i.landmark[12].y*480)
                y9 = int(i.landmark[9].y*480)

                if (index_raised(yi,y9)):

                    if not(var_inits):
                        xii,yii = x,y
                        var_inits = True
                    cv2.line(frame,(xii,yii),(x,y),(92,76,255),thick)

                else:
                    if var_inits:
                        cv2.line(mask,(xii,yii),(x,y),0,thick)
                        var_inits = False


            elif curr_tool == "draw":
                xi, yi = int(i.landmark[12].x*640), int(i.landmark[12].y*480)
                y9  = int(i.landmark[9].y*480)
                    
                if (index_raised(yi,y9)):

                    if not(var_inits):
                        var_inits = True
                    if (index_raised(yi,y9) == False):
                        cv2.line(mask, (prevx, prevy), (x, y), 0, thick)
                    prevx = x
                    prevy = y
                          
                        
                        
                else:
                    if var_inits:
                        var_inits = False
                    cv2.line(mask, (prevx, prevy), (x, y), 0, thick)
                    #curr_tool = "Select"
                    prevx = x
                    prevy = y


            
            elif curr_tool == "rectangle":
                xi, yi = int(i.landmark[12].x*640), int(i.landmark[12].y*480)
                y9  = int(i.landmark[9].y*480)

                if index_raised(yi, y9):
                    if not(var_inits):
                        xii, yii = x, y
                        var_inits = True

                    cv2.rectangle(frame, (xii, yii), (x, y), (0,255,255), thick)

                else:
                    if var_inits:
                        cv2.rectangle(mask, (xii, yii), (x, y), 0, thick)
                        var_inits = False
                        

            elif curr_tool == "circle":
                xi, yi = int(i.landmark[12].x*640), int(i.landmark[12].y*480)
                y9  = int(i.landmark[9].y*480)

                if index_raised(yi, y9):
                    if not(var_inits):
                        xii, yii = x, y
                        var_inits = True

                    cv2.circle(frame, (xii, yii), int(((xii-x)**2 + (yii-y)**2)**0.5), (255,255,0), thick)

                else:
                    if var_inits:
                        cv2.circle(mask, (xii, yii), int(((xii-x)**2 + (yii-y)**2)**0.5), (0,255,0), thick)
                        var_inits = False


                        

            elif (curr_tool == "erase"):

                xi,yi = int(i.landmark[12].x*640),int(i.landmark[12].y*480)
                y9 = int(i.landmark[9].y*480)

                if (index_raised(yi,y9) == True):

                    cv2.circle(frame,(x,y),30,(92,255,255),-1)
                    cv2.circle(mask,(x,y),30,255,-1)

            elif curr_tool == "save" :


                xi,yi = int(i.landmark[12].x*640),int(i.landmark[12].y*480)
                y9 = int(i.landmark[9].y*480)

                if (index_raised(yi,y9)):

                    if not(var_inits):
                        
                        var_inits = True
                    

                else:
                    if var_inits:
                        fileNum+=1
                        im = Image.fromarray(mask)
                        fileName = "screenshot_{}.jpg".format(fileNum)
                        
                        im.save(fp+"\\"+fileName)
                        print("saved ")
                        curr_tool = "Select"
                        var_inits = False
                        
                        
                

    #frame = cv2.bitwise_and(frame, frame, mask=mask)
    #frame[:max_y,ml:max_x] = cv2.addWeighted(img,0.7,frame[:max_y,ml:max_x],0.3,0)


    cv2.putText(frame, curr_tool, (500,35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)


    if (rec_count == True):
        out.write(frame)
    
    """ DISPLAYING BOTH THE WINDOWS """
    
    cv2.imshow('Test video', frame)
    cv2.imshow('mask',mask)


    
    if (exit_variable == True):
        break
    


    if (cv2.waitKey(1)==ord('q')):
        print('wait')
        video.release()
        cv2.destroyAllWindows()
        break
    

video.release()
out.release()
cv2.destroyAllWindows()
  
c=0
inn=0
if (fileNum > 0 ) :

    with open(fp+"\\output.pdf", "wb") as f:
        os.chdir(fp)
        
        f.write(img2pdf.convert([i for i in os.listdir(os.getcwd()) if i.endswith(".jpg")]))
              
    for file in os.listdir(os.getcwd()):
        if file.endswith('.jpg'):
            os.remove(file)




saved 
saved 
